In [1]:
!pip install polars

  Using cached polars-1.35.2-py3-none-any.whl.metadata (10 kB)
  Using cached polars_runtime_32-1.35.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached polars-1.35.2-py3-none-any.whl (783 kB)
Using cached polars_runtime_32-1.35.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [polars]2m1/2 [polars]


In [5]:
import polars as pl
import os
import sys

sys.path.append("../src")
from grouping_data import clean_types, group_reviews_by_product

In [6]:
s3_path = "s3://amazon-electronics-dataset/Preprocessed_dataset/electronics_clean.csv"

df_lazy = pl.scan_csv(
    s3_path,
    schema_overrides={"is_verified": pl.Utf8, "product_id": pl.Utf8},
    infer_schema_length=0,
    ignore_errors=True
)

In [7]:
df_head = df_lazy.collect()
df_head

reviewer_id,product_id,original_review,user_summary,rating,helpful_votes,is_verified,review_date,review_timestamp,product_style,cleaned_review,review_count
str,str,str,str,str,str,str,str,str,str,str,str
"""AAP7PPBU72QFM""","""0151004714""","""This is the best novel I have …","""A star is born""","""5.0""","""67""","""True""","""1999-09-18""","""937612800""",""" Hardcover""","""this is the best novel i have …","""5"""
"""A2E168DTVGE6SV""","""0151004714""","""Pages and pages of introspecti…","""A stream of consciousness nove…","""3.0""","""5""","""True""","""2013-10-23""","""1382486400""",""" Kindle Edition""","""pages and pages of introspecti…","""5"""
"""A1ER5AYS3FQ9O3""","""0151004714""","""This is the kind of novel to r…","""I'm a huge fan of the author a…","""5.0""","""4""","""False""","""2008-09-02""","""1220313600""",""" Paperback""","""this is the kind of novel to r…","""5"""
"""A1T17LMQABMBN5""","""0151004714""","""What gorgeous language! What a…","""The most beautiful book I have…","""5.0""","""13""","""False""","""2000-09-04""","""968025600""",""" Hardcover""","""what gorgeous language what an…","""5"""
"""A3QHJ0FXK33OBE""","""0151004714""","""I was taken in by reviews that…","""A dissenting view--In part.""","""3.0""","""8""","""True""","""2000-02-04""","""949622400""",""" Hardcover""","""i was taken in by reviews that…","""5"""
…,…,…,…,…,…,…,…,…,…,…,…
"""A1HKXEX8BEQC2E""","""B01HJH40WU""","""not holding up over time :(""","""not holding up over time :(""","""3.0""","""0""","""True""","""2017-06-26""","""1498435200""",null,"""not holding up over time""","""7"""
"""A33MAQA919J2V8""","""B01HJH40WU""","""These seem like quality USB ca…","""Four Stars""","""4.0""","""0""","""True""","""2017-03-21""","""1490054400""",null,"""these seem like quality usb ca…","""7"""
"""A1AKHSCPD1BHM4""","""B01HJH40WU""","""Works great, love the longer c…","""Nice long cord""","""4.0""","""0""","""True""","""2017-01-09""","""1483920000""",null,"""works great love the longer co…","""7"""


In [10]:
#Clean dtypes
df=clean_types(df_head)
df.head()

reviewer_id,product_id,original_review,user_summary,rating,helpful_votes,is_verified,review_date,review_timestamp,product_style,cleaned_review,review_count
str,str,str,str,f64,i64,i64,str,i64,str,str,i64
"""AAP7PPBU72QFM""","""0151004714""","""This is the best novel I have …","""A star is born""",5.0,67,1,"""1999-09-18""",937612800,""" Hardcover""","""this is the best novel i have …",5
"""A2E168DTVGE6SV""","""0151004714""","""Pages and pages of introspecti…","""A stream of consciousness nove…",3.0,5,1,"""2013-10-23""",1382486400,""" Kindle Edition""","""pages and pages of introspecti…",5
"""A1ER5AYS3FQ9O3""","""0151004714""","""This is the kind of novel to r…","""I'm a huge fan of the author a…",5.0,4,0,"""2008-09-02""",1220313600,""" Paperback""","""this is the kind of novel to r…",5
"""A1T17LMQABMBN5""","""0151004714""","""What gorgeous language! What a…","""The most beautiful book I have…",5.0,13,0,"""2000-09-04""",968025600,""" Hardcover""","""what gorgeous language what an…",5
"""A3QHJ0FXK33OBE""","""0151004714""","""I was taken in by reviews that…","""A dissenting view--In part.""",3.0,8,1,"""2000-02-04""",949622400,""" Hardcover""","""i was taken in by reviews that…",5


In [11]:
#Grouping data
grouped_data=group_reviews_by_product(df_head)
grouped_data.head()

product_id,all_reviews,all_user_summaries,avg_rating,review_count,total_helpful_votes,dominant_style,oldest_review_timestamp,newest_review_timestamp
str,str,str,str,u32,str,str,str,str
"""B003MXJ0QW""","""just what i was looking for th…","""good cable wrong product po…",null,5,null,null,"""1354233600""","""1421798400"""
"""B0007RA6NW""","""this radio is very light easy …","""Great portable radio WOW!!!! …",null,16,null,null,"""1120089600""","""1447027200"""
"""B00ZQUMH1Y""","""a useful item to carry when tr…","""A good item Four Stars Must …",null,9,null,null,"""1510704000""","""1529625600"""
"""B0154NRMCG""","""the twist lock connectors are …","""Incorrectly Wired and Poor Qua…",null,18,null,null,"""1462320000""","""1522454400"""
"""B00OHDONOA""","""i bought this mp3 player to us…","""Great MP3 with amazing battery…",null,42,null,null,"""1429747200""","""1466294400"""


In [13]:
#Save to S3
local_output = "product_grouped.csv"
s3_output = "s3://amazon-electronics-dataset/grouping_dataset/product_grouped.csv"

grouped_data.write_csv(local_output)

os.system(f"aws s3 cp {local_output} {s3_output}")

print("Saved grouped dataset to:", s3_output)

upload: ./product_grouped.csv to s3://amazon-electronics-dataset/grouping_dataset/product_grouped.csv
Saved grouped dataset to: s3://amazon-electronics-dataset/grouping_dataset/product_grouped.csv
